In [1]:
import pandas as pd
from datetime import date, datetime
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import glob

In [2]:
file_path = "data/Quarterly files"
csv_files = glob.glob(os.path.join(file_path, '*.csv'))
print(csv_files)

['data/Quarterly files/2023 Profit and Loss Q3.csv']


In [3]:
latest_file = max(csv_files, key=os.path.getmtime)

In [4]:
df = pd.read_csv(latest_file, index_col = False)

In [5]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,BJ WIN HOSPITALITY INC.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Profit and Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"For the Nine Months Ending September 30, 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Jul. 2023,Aug. 2023,Sept. 2023,Year to Date,NaN,NaN,NaN
